In [ ]:
!rm -rf /content/te_sql_summary_2018/

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [ ]:
import findspark
findspark.init()
spark_url = 'local'

import os
import json
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

In [ ]:
import zipfile
import os

input_path = "/content/2023.zip"
output_folder = "/content/2023_extracted"

os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(input_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

print(f"Files extracted to: {output_folder}")

Files extracted to: /content/2023_extracted


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

file_path = "/content/2023_data_final.csv"
output_path = "/content/te_sql_2023"

df = spark.read.csv(file_path, header=True, inferSchema=True)

columns_to_drop = ["filename", "srctype", "prism_aggregationType", "subtypeDescription", "subject_area_name", "subject_area_code", "source_id", "subtype", "dc_publisher"]
df_dropped = df.drop(*columns_to_drop)

df_renamed = (df_dropped
              .withColumnRenamed("subject_area_abbrev", "Subject")
              .withColumnRenamed("dc_identifier", "Identifier")
              .withColumnRenamed("eid", "EID")
              .withColumnRenamed("dc_title", "Title")
              .withColumnRenamed("publicationName", "Publication Name")
              .withColumnRenamed("prism_coverDate", "Publication Date")
              .withColumnRenamed("prism_doi", "DOI")
              .withColumnRenamed("citedby_count", "Cited By Count")
              .withColumnRenamed("affiliation_country", "Affiliation Country"))


columns_order = ["Subject","Identifier","EID","Title","Publication Name","Publication Date","DOI","Cited By Count","Affiliation Country"]
df_reordered = df_renamed.select(columns_order)

df_with_year = df_reordered.withColumn("Year", lit(2023))

df_sorted = df_with_year.orderBy("Subject", "Identifier")



df_sorted.write.mode("overwrite").csv(output_path, header=True)
print("Data has been saved to:", output_path)


Data has been saved to: /content/te_sql_2023


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Remove Header Completely").getOrCreate()

file_path = "/content/te_sql_2023/te_sql_2023.csv"
output_path = "/content/te_sql_2023_no_header"


df = spark.read.csv(file_path, header=False, inferSchema=True)

df_with_index = df.rdd.zipWithIndex().filter(lambda row: row[1] > 0).map(lambda row: row[0])

df_new = spark.createDataFrame(df_with_index)

df_new.show()

df_new.write.mode("overwrite").csv(output_path, header=False)
print("Data with the header completely removed has been saved to:", output_path)


+----+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---+-----------------+----+
| _c0|                 _c1|               _c2|                 _c3|                 _c4|                 _c5|                 _c6|_c7|              _c8| _c9|
+----+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---+-----------------+----+
|AGRI|SCOPUS_ID:8512347...|2-s2.0-85123470593|Factors affecting...|Food Science and ...|2023-04-01T00:00:...|10.1177/108201322...|  1|         Thailand|2023|
|AGRI|SCOPUS_ID:8513549...|2-s2.0-85135493764|Non-antibiotic ap...|Reviews in Aquacu...|2023-01-01T00:00:...|   10.1111/raq.12721| 11|         Malaysia|2023|
|AGRI|SCOPUS_ID:8513549...|2-s2.0-85135493764|Non-antibiotic ap...|Reviews in Aquacu...|2023-01-01T00:00:...|   10.1111/raq.12721| 11|         Thailand|2023|
|AGRI|SCOPUS_ID:8513549...|2-s2.0-85135493764|Non-an

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

spark = SparkSession.builder.appName("Count Unique Titles").getOrCreate()

file_path = "/content/te_sql_2023/te_sql_2023.csv"
output_path = "/content/te_sql_summary_2023"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df_counted = (df.groupBy("Year", "Affiliation Country", "Subject")
                .agg(countDistinct("Title").alias("Number of Documents"))
                .withColumnRenamed("Affiliation Country", "Country")
                .withColumnRenamed("Subject", "Subject Area"))

df_result = df_counted.select("Country", "Year", "Subject Area", "Number of Documents").orderBy("Country", "Year", "Subject Area")

df_result.show()

df_result.write.mode("overwrite").csv(output_path, header=True)
print("Summary data has been saved to:", output_path)
row_count = df_result.count()
print(f"Number of rows: {row_count}")


+-----------+----+------------+-------------------+
|    Country|Year|Subject Area|Number of Documents|
+-----------+----+------------+-------------------+
|Afghanistan|2023|        MEDI|                  1|
|    Albania|2023|        MEDI|                  3|
|    Algeria|2023|        MEDI|                  1|
|    Algeria|2023|        PSYC|                  1|
|  Argentina|2023|        AGRI|                  1|
|  Argentina|2023|        MEDI|                 16|
|  Argentina|2023|        MULT|                  1|
|  Argentina|2023|        NEUR|                  1|
|  Argentina|2023|        PSYC|                  2|
|  Argentina|2023|        SOCI|                  1|
|    Armenia|2023|        ENGI|                  5|
|    Armenia|2023|        MEDI|                  4|
|    Armenia|2023|        PHYS|                 25|
|  Australia|2023|        AGRI|                  6|
|  Australia|2023|        BIOC|                 10|
|  Australia|2023|        BUSI|                  6|
|  Australia